<a href="https://colab.research.google.com/github//pawel717-SCGE/lab1/blob/master/lab1-SCGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>
Szeregi czasowe
</h1>
<h2>
Sprawozdanie z laboratorium
</h2>
</center>

<p style="text-align: right">
    <b><i>Wykonali:</i></b><br>
<i>Paweł Suchanicz</i><br>
<i>Elżbieta Dziedzic</i>
</p>


# LAB 1

## Dane z rozkładu normalnego

### Generowanie danych o rozkładzie normalnym

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns

mu = 1.0
sigma = 2.0
sample = np.random.normal(mu, sigma, 10000)

Wygenerowano próbkę z rozkładu normalnego o parametrach μ=1.0 i σ=2.0. Liczebność próbki wynosi 10000.

In [ ]:
plt.title("Wykres danych z rozkładu normalnego o parametrach μ=1.0 i σ=2.0")
plt.plot(sample)

### Standaryzacja danych

In [ ]:
# funkcja do standryzacji próbki
def standarize(sample):
    return (sample - sample.mean())/(sample.std())

sample_standarized = standarize(sample)
plt.title("Wykres danych z rozkładu normalnego o parametrach μ=1.0 i σ=2.0 po standaryzacji")
plt.plot(sample_standarized)

### Tworzenie histogramu

In [ ]:
# funkcja do tworzenia histogramu z podnych danych (sample)
def plot_histogram(sample, title=''):
    plt.title(title)
    count, bins, patches = plt.hist(sample, 20, density=True)
    sns.kdeplot(sample)
    plt.show()
    
plot_histogram(sample, "Histogram z próbki bez standaryzacji")
plot_histogram(sample_standarized, "Histogram z próbki po standaryzacji")

### Histogram w skali logarytmicznej

In [ ]:
# funkcja do tworzenia histogramu w skali logarytmicznej z podanej próbki (sample)
def plot_log_histogram(sample, title=''):
    plt.title(title)
    plt.yscale('log', nonposy='clip')
    count, bins, patches = plt.hist(sample, 20, density=True)
    plt.show()
    
plot_log_histogram(sample_standarized, 'Histogram z próbki po standaryzacji w skali logarytmicznej')

 ### Dystrybuanta

In [ ]:
def plot_cumulative_distribution(sample, sigma, mu, title=''):
    count, bins, patches = plt.hist(sample, 20, density=True, cumulative=True,histtype='step', label='Empirical')
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
         np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    y = y.cumsum()
    y /= y[-1]
    plt.title(title)
    plt.plot(bins, y, 'k--', linewidth=1.5, label='Theoretical')
    plt.legend(loc='upper left')
    plt.show()

plot_cumulative_distribution(sample, sigma=sigma, mu=mu, title="Wykres dystrybuanty dla próbki bez standaryzacji")
plot_cumulative_distribution(sample_standarized, sigma=1.0, mu=0, title="Wykres dystrybuanty dla próbki po standaryzacji")

Na wykresach wartości empiryczne dystrybuanty pokrywają się z teoretycznymi.

### Autokorelacja 

In [ ]:
pd.plotting.autocorrelation_plot(sample_standarized)

Używamy wykresu autokorelacji, aby sprawdzić czy w próbce danych występują zależności między danymi dla różnych opóźnień czasowych.

Na wykresie wszystkie autokorelacje wynoszą oscylują bardzo blisko w granicach 0, więc analizowany rozkład jest losowy, dane układają się w sposób przypadkowy.



## Dane Giełdowe

Wczytanie danych giełdowych (notowania S&P500 https://finance.yahoo.com/quote/%5EGSPC/):

In [ ]:
import pandas as pd
df_gspc = pd.read_csv('gspc.csv', index_col='Date')
df_gspc

In [ ]:
df_gspc.describe()

In [ ]:
df_close = df_gspc['Adj Close']
df_close.plot(title='Wykres cen zamknięcia S&P 500 na przestrzeni lat')

Z wykresu wnioskujemy, że dane posiadają trend rosnący.

Nastepnie liczymy zwroty jako różnicę obecnej i poprzedniej ceny zamknięcia.

In [ ]:
rets = df_close / df_close.shift(1) - 1
rets.plot(title='Wykres zmian w cenach zamknięcia S&P500 na przestrzeni lat')

Wyliczamy także logarytmiczne stopy zwrotu:

In [ ]:
df_change = pd.DataFrame(df_close).apply(lambda x: np.log(x) - np.log(x.shift(1))) 
df_change.describe()

In [ ]:
df_change.plot(title='Wykres logarytmiczny stóp zwrotu S&P500 na przestrzeni lat')

In [ ]:
import scipy.stats as stats
fig, ax = plt.subplots(figsize=(10, 6))


shape, loc, scale = stats.lognorm.fit(df_close) 
x = np.linspace(df_close.min(), df_close.max(), len(df_close))
pdf = stats.lognorm.pdf(x, shape, loc=loc, scale=scale) 
label = 'mean=%.4f, std=%.4f, shape=%.4f' % (loc, scale, shape)
ax.set_title("Histogram stóp zwrotu S&P500")
ax.hist(df_close, bins=30, density=True)
ax.plot(x, pdf, 'r-', lw=2, label=label)
ax.legend(loc='best')

In [ ]:
values = df_change['Adj Close'][1:]  # skip first NA value
x = np.linspace(values.min(), values.max(), len(values))

loc, scale = stats.norm.fit(values)
param_density = stats.norm.pdf(x, loc=loc, scale=scale)
label = 'mean=%.4f, std=%.4f' % (loc, scale)

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_title("Histogram logarytmicznych stóp zwrotu S&P500")
ax.hist(values, bins=30, density=True)
ax.plot(x, param_density, 'r-', label=label)
ax.legend(loc='best')

In [ ]:
values = df_change['Adj Close'][1:]  # skip first NA value
x = np.linspace(values.min(), values.max(), len(values))

loc, scale = stats.norm.fit(values)
param_density = stats.norm.cdf(x, loc=loc, scale=scale)
label = 'mean=%.4f, std=%.4f' % (loc, scale)

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_title("Dystrybuanta logarytmicznych stóp zwrotu S&P500")
ax.hist(values, bins=30, density=True, cumulative=True, histtype='step')
ax.plot(x, param_density, 'r-', label=label)
ax.legend(loc='best')

### Autokorelacja

In [ ]:
pd.plotting.autocorrelation_plot(df_close);

Z wykresu wynika, że dla danych giełdowych występują zależności. Najsilniejsze autokorelacje znajdują się na początku oraz w okolicy 15000.

In [ ]:
pd.plotting.autocorrelation_plot(rets[1:]);

Autokorelacje na wykresie wynoszą 0. W szeregu z wartościami zwrotu nie są widoczne korelacje.

In [ ]:
pd.plotting.autocorrelation_plot(df_change['Adj Close'][1:]);

Autokorelacje na wykresie wynoszą 0. Także w szeregu z wartościami logarytmicznymi zwrotu nie są widoczne korelacje.

# LAB 2
## Notowania kryptowalut

Wczytanie danych dotyczących nototwań kryptowaluty Bitcoin(BTC) w stosunku do dolara ameykańskiego(USD):

In [ ]:
import pandas as pd
df_btcusd = pd.read_csv('Bitfinex_BTCUSD_1h.csv', index_col='Date')
df_btcusd

In [ ]:
df_btcusd.describe()

In [ ]:
df_btcusd_close = df_btcusd['Close']
df_btcusd_close.plot(title='Wykres cen zamknięcia notowań BTC/USD na przestrzeni lat')

Na wykresie znajduje się kilka lokalnych trendów.

Wyliczamy fluktuacje (zmiany) w cenach zamknięcia notowań jako różnica wartości obecnej i poprzeniego notowania.

In [ ]:
btdusd_rets = df_btcusd_close / df_btcusd_close.shift(1) - 1
btdusd_rets.plot(title='Wykres zmian w cenach zamknięcia notowań BTC/USD na przestrzeni lat')

Wyliczamy także logarytmiczne stopy zwrotu:

In [ ]:
df_btdusd_change = pd.DataFrame(df_btcusd_close).apply(lambda x: np.log(x) - np.log(x.shift(1))) 
df_btdusd_change.describe()
df_btdusd_change.plot(title='Wykres logarytmiczny stóp zwrotu BTC/USD na przestrzeni lat')

In [ ]:
import scipy.stats as stats
fig, ax = plt.subplots(figsize=(10, 6))


loc, scale = stats.norm.fit(df_btcusd_close) 
x = np.linspace(df_btcusd_close.min(), df_btcusd_close.max(), len(df_btcusd_close))
pdf = stats.norm.pdf(x, loc=loc, scale=scale) 
label = 'mean=%.4f, std=%.4f, shape=%.4f' % (loc, scale, shape)
ax.set_title("Histogram stóp zwrotu BTC/USD")
ax.hist(df_btcusd_close, bins=30, density=True)
ax.plot(x, pdf, 'r-', lw=2, label=label)
ax.legend(loc='best')

Prawa strona histogramu częściowo układa się w sposób zbliżony do rozkładu normalnego, jego lewa strona zupełnie od niego odbiega, występują maksima wykraczające poza rozkład normalny.

In [ ]:
values = df_btdusd_change['Close'][1:]  # skip first NA value
x = np.linspace(values.min(), values.max(), len(values))

loc, scale = stats.norm.fit(values)
param_density = stats.norm.pdf(x, loc=loc, scale=scale)
label = 'mean=%.4f, std=%.4f' % (loc, scale)
ax.set_title("Histogram loagrytmicznych stóp zwrotu BTC/USD")
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(values, bins=30, density=True)
ax.plot(x, param_density, 'r-', label=label)
ax.legend(loc='best')

Histogram jest zbliżony do rozkładu normalnego, jednak bardziej przechylony na prawo i wykraczający poza maksumum rozkładu.

In [ ]:
values = df_btdusd_change['Close'][1:]  # skip first NA value
x = np.linspace(values.min(), values.max(), len(values))

loc, scale = stats.norm.fit(values)
param_density = stats.norm.cdf(x, loc=loc, scale=scale)
label = 'mean=%.4f, std=%.4f' % (loc, scale)

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_title("Dystrybuanta logarytmicznych stóp zwrotu BTC/USD")
ax.hist(values, bins=30, density=True, cumulative=True, histtype='step')
ax.plot(x, param_density, 'r-', label=label)
ax.legend(loc='best')

Dystrybuanta empiryczna jest zbliżona do teoretycznej. W górnej części wykresu najbardziej od niej odbiega.

### Autokorelacja

In [ ]:
pd.plotting.autocorrelation_plot(df_btcusd['Close'])

Z wykresu autokorelacji wynika, że w szeregu występują zależności. 

In [ ]:
pd.plotting.autocorrelation_plot(df_btdusd_change['Close'][1:])

**Autokorelacja dla wymieszanych danych**

Mieszając dane w sposób losowy usuwamy korelację między danymi, dlatego w wyniku otrzymany wykres autokorelacji jest linią prostą y=0 (autokorelacja wynosi 0)

In [ ]:
shuffled = sklearn.utils.shuffle(df_btcusd)
pd.plotting.autocorrelation_plot(shuffled['Close'])

## Data-puzzle

Wczytujemy dane z pliku:

In [ ]:
import pandas as pd
df_data_puzzle = pd.read_csv('data_puzzle.dat', header=None)
df_data_puzzle

In [ ]:
df_data_puzzle.describe()

In [ ]:
df_data_puzzle.plot(title="Wykres danych data_puzzle")

Dane na wykresie przejawiają sezonowość.

Następnie standaryzujemy dane i wyliczamy stopy zwtotu jako różnicę obecnej wartości i poprzedniej:

In [ ]:
df_data_puzzle = standarize(df_data_puzzle)
data_puzzle_rets = df_data_puzzle / df_data_puzzle.shift(1) - 1
data_puzzle_rets.plot(label='return')

In [ ]:
data_puzzle_rets.describe()

Wyliczamy też logarytmiczne stopy zwrotu:

In [ ]:
import numpy as np
df_data_puzzle_change = df_data_puzzle.apply(lambda x: np.log(x) - np.log(x.shift(1))) 
df_data_puzzle_change.plot(title="Logarytmiczne stopy zwrotu dla danych data_puzzle")

In [ ]:
import scipy.stats as stats
fig, ax = plt.subplots(figsize=(10, 6))


loc, scale = stats.norm.fit(data_puzzle_rets) 
x = np.linspace(data_puzzle_rets.min(), data_puzzle_rets.max(), len(data_puzzle_rets))
pdf = stats.norm.pdf(x, loc=loc, scale=scale) 
label = 'mean=%.4f, std=%.4f, shape=%.4f' % (loc, scale, shape)
ax.set_title("Histogram stóp zwrotu BTC/USD")
ax.hist(data_puzzle_rets, bins=30, density=True)
ax.plot(x, pdf, 'r-', lw=2, label=label)
ax.legend(loc='best')

Na wykresie pojawiają się skupiska o podobnej szerokości.

In [ ]:
df_data_puzzle_change.describe()

Obliczamy autokorelację dla danych data_puzzle

In [ ]:
pd.plotting.autocorrelation_plot(df_data_puzzle[0])

W szeregu znajdują się zależności. Wykres autokorelacji jest podobny do wykresu drgań oscylatora harmonicznego tłumionego.

# LAB 4
## Transformata Fouriera i widmo mocy

https://ipython-books.github.io/101-analyzing-the-frequency-components-of-a-signal-with-a-fast-fourier-transform/

In [ ]:
import scipy as sp
import scipy.fftpack
data_puzzle_fft = sp.fftpack.fft(df_data_puzzle)

In [ ]:
data_puzzle_psd = np.abs(data_puzzle_fft) ** 2

In [ ]:
plt.plot(data_puzzle_fft)

In [ ]:
plt.plot(data_puzzle_psd)